# Introduction

## Using Jupyter

Jupyter is a front end to the [IPython](https://ipython.org) interactive shell, and offers IDE like features.  It is separated into two main types of cell: [Markdown](https://en.wikipedia.org/wiki/Markdown) cells (such as this one) which allows markdown or HTML code to be written, and code cells like the next one which can be run in real time.

To execute code in a cell, press `Crtl` + `Enter`, click on the `[ > ]Run` button in the main menu, or press `Shift` + `Enter` if you wish to execute the code and then move on to a new cell (creating it if it does not already exist).

## Setup

MongoDB is a NoSQL database, which has a core API in JavaScript, and a series of other APIs in different languages.  The one we are going to use is the Python API, [PyMongo](https://api.mongodb.com/python/current/).  MongoDB instance on your VM is already started by default.

PyMongo is a package that contains tools to work with MongoDB from Python. We have installed it in the VM provided to you. 

This imports the `MongoClient` class from the pymongo module, which we will use to deal with all our connections from.  We're connecting to our localhost, which is listening on port 27017. There are more options, the documentation for the formatting of the connection string is at https://docs.mongodb.com/manual/reference/connection-string/.

In [2]:
!pip install pymongo

    100% |████████████████████████████████| 409kB 1.5MB/s ta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

If you do not get any errors, you have confirmed PyMongo library has been successfully installed&configured in the VM. We will now check to see whether the connection is correct.  The following code calls a function which returns a list of all current databases.  If your Mongo instance is still empty, it should be something like `['admin', 'local']`.

In [19]:
client.list_database_names()

['admin',
 'client-app-1',
 'client-app-2',
 'conFusion',
 'local',
 'mongo_lab',
 'mycustomers']

Next, we are going to create a database object `db`, which is a property of the `client` object.  MongoDB is schemaless, and so accessing a database like this will create the database if it does not already exist.

A database can be accessed by using "dot" notation (i.e., `client.dbname`), or dictionary notation (i.e., `client['dbname']`).  This also applies to making collections

Create a database called `test` in a variable called `db`.  Using that variable, create a collection called `test_collection` with a variable called `collection` as follows.  Run the code in the following cell (there should not be any output)

In [14]:
# Create database and collection objects for convenience
db = client.mongo_lab
collection = db.test_collection

# Using MongoDB

## Inserting data

MongoDB data is stored as BSON (binary JSON), which is essentially JSON with some additional optimisations, so the way to insert data is as a JSON object.  For Python, you can use a `dict` or a `list` for this, and then call either `insert_one` or `insert_many` on the collection.  

In [18]:
# Create an object and insert into the `test_collection`
single_obj = {'name': 'Amber', 'star_sign': 'Capricorn', 'favourite_song': 'The Load-Out'}
collection.insert_one(single_obj)
single_obj_2 = {'name': 'Huw', 'star_sign': 'Libra', 'favourite_song': 'The masses against the classes'}
collection.insert_one(single_obj_2)
single_obj_3 = {'name': 'Robert', 'star_sign': 'Leo', 'favourite_song': 'Bad day'}
collection.insert_one(single_obj_3)

We will look at querying data in more detail below, but for now, to see whether the object got successfully inserted into the collection, run the code below.  This will always return the first instance which matches the query.  You will notice that even though we didn't specify `_id` one got added already.  This is a unique identifier for the document in the collection

In [20]:
collection.find_one() # returns a single document matching the query condition

{'_id': ObjectId('5c09b5f9651cff367fe07129'),
 'favourite_song': 'The Load-Out',
 'name': 'Amber',
 'star_sign': 'Capricorn'}

In [21]:
collection.find_one({'name':'Robert'})

{'_id': ObjectId('5c09b5f9651cff367fe0712b'),
 'favourite_song': 'Bad day',
 'name': 'Robert',
 'star_sign': 'Leo'}

Remember, for MongoDB, you do not have to specify a schema or create a collection, it will be created automatically.  You don't need to keep to the same layout, but can have entirely different objects.  Consider the following: 

In [22]:
from datetime import datetime
obj1 = {'Meaning of life': 42}
obj2 = {'ABC': 'DEF', 'time': datetime.now()}
collection.insert_one(obj1)
collection.insert_one(obj2)

We can also use the `insert_many`, which accepts a list of dicts.  In the cell below, create a list of dicts called `many_objects`, and call the `insert_many` function.  The code below that will iterate over all the documents in the database.

In [26]:
from pprint import pprint

In [27]:
# YOUR CODE HERE

list_of_dicts = [
    {
        "name": "Mini",
        "age": 23,
        "planet": "unknown"
    },
    {
        "name": "Paddy",
        "age": 25
    },
    {
        "name": "Saurav",
        "planet": "Earth"
    }
]

# collection.insert_many(list_of_dicts)


#See what has been inserted into the collection
for doc in collection.find():
    pprint(doc)

{'_id': ObjectId('5c09b5f9651cff367fe07129'),
 'favourite_song': 'The Load-Out',
 'name': 'Amber',
 'star_sign': 'Capricorn'}
{'_id': ObjectId('5c09b5f9651cff367fe0712a'),
 'favourite_song': 'The masses against the classes',
 'name': 'Huw',
 'star_sign': 'Libra'}
{'_id': ObjectId('5c09b5f9651cff367fe0712b'),
 'favourite_song': 'Bad day',
 'name': 'Robert',
 'star_sign': 'Leo'}
{'Meaning of life': 42, '_id': ObjectId('5c09b642651cff367fe0712c')}
{'ABC': 'DEF',
 '_id': ObjectId('5c09b642651cff367fe0712d'),
 'time': datetime.datetime(2018, 12, 6, 23, 52, 34, 776000)}
{'_id': ObjectId('5c09b741651cff367fe0712e'),
 'age': 23,
 'name': 'Mini',
 'planet': 'unknown'}
{'_id': ObjectId('5c09b741651cff367fe0712f'), 'age': 25, 'name': 'Paddy'}
{'_id': ObjectId('5c09b741651cff367fe07130'),
 'name': 'Saurav',
 'planet': 'Earth'}


## Importing and querying data

For this part of the exercise, we will use a sample dataset provided by Mongo  for a documentation tutorial.  The following cell runs the `mongoimport` command, which is a Unix command which comes with Mongo for importing data. We will need to run a bash command in the next cell first.  This uses the Jupyter \"magics\", and requires that the first line include `%%bash`. The code does the following:

- Download the JSON file from the url, and save as ./primer-dataset.json
- Import into the `test` database into the collection `restaurants` whilst dropping any collection which already exists from the file ./primer-dataset.json
- Deletes the file.

 Click on the following cell, and execute it:

In [28]:
%%bash
# Use wget to download the data
wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
# mongoimport is the Mongo command to import data.  
# It specifies the database, collection and format, and import file
# --drop means it's going to drop any collection with the same name which already exists
mongoimport --db mongo_lab --collection restaurants --drop --file ./primer-dataset.json
# Delete the JSON file we just downloaded
rm ./primer-dataset.json

--2018-12-06 23:57:58--  https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11874761 (11M) [text/plain]
Saving to: ‘primer-dataset.json’

     0K .......... .......... .......... .......... ..........  0% 1.78M 6s
    50K .......... .......... .......... .......... ..........  0% 3.40M 5s
   100K .......... .......... .......... .......... ..........  1% 7.36M 4s
   150K .......... .......... .......... .......... ..........  1% 2.46M 4s
   200K .......... .......... .......... .......... ..........  2% 3.91M 4s
   250K .......... .......... .......... .......... ..........  2% 1.87M 4s
   300K .......... .......... .......... .......... ..........  3% 4.70M 4s
   350K .......... .......... .......... .......... .

Change the variable `collection` to refer to the new collection `restaurants`, and inspect the general format of the data by adding the code below to find the first record of the collection:

In [29]:
# YOUR CODE HERE

collection = db.restaurants

We saw the [`collection.find()`](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find) function earlier to return all the documents we inserted into our `test` collection.  Without any arguments, `find()` will return a cursor of all the available documents from in the collection.  To refine queries, however, the search can be filtered by the addition of a first parameter.

The `filter` parameter is a dict, which searches for the documents where `key` = `value` where the dict is of the form `{key: value}`.  For example, to find all bakeries in the city, we would do the following query:   

**WARNING** Unlike the Mongo command line interface, if you try and print the output of a `find()` query, it will continue to output all results until it has finished.  This can cause the browser to crash, particularly if it is a particularly large query set.

Using `find().count()` is a useful way of checking how many a result will return, and `find_one()` to see the general structure of a result. If you use `find()`, make sure you either include the `limit` argument, or have a counter or other condition to break out of your printing loop!

In [31]:
for doc in collection.find({'cuisine': 'Bakery'}).limit(5):
    pprint(doc)

{'_id': ObjectId('5c09b78cd8c8b5898b20c861'),
 'address': {'building': '1007',
             'coord': [-73.856077, 40.848447],
             'street': 'Morris Park Ave',
             'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
             'grade': 'A',
             'score': 2},
            {'date': datetime.datetime(2013, 9, 11, 0, 0),
             'grade': 'A',
             'score': 6},
            {'date': datetime.datetime(2013, 1, 24, 0, 0),
             'grade': 'A',
             'score': 10},
            {'date': datetime.datetime(2011, 11, 23, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2011, 3, 10, 0, 0),
             'grade': 'B',
             'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}
{'_id': ObjectId('5c09b78cd8c8b5898b20c879'),
 'address': {'building': '120',
             'coord': [-73.9998042, 40.7251256],
 

Noted, count() is deprecated in the MongoDb drivers compatible with the 4.0features, as a result, we should use countDocuments() in the following exercises. (for more information, check https://docs.mongodb.com/manual/reference/method/db.collection.count/ and http://api.mongodb.com/python/current/changelog.html)

In [32]:
collection.count_documents({'cuisine': 'Bakery'})

691

A filter can have as many conditions as you like, and will assume that you are using an AND condition, unless you specify otherwise (as below).  In the cell below, write a query to return the number/count of all the establishments with a cuisine of `Hamburgers` in the borough of Manhattan.

In [33]:
# YOUR CODE HERE
# All establishments with: 
# * a cuisine of 'Hamburgers' 
# * in the borough of 'Manhattan

collection.count_documents({'cuisine': 'Hamburgers', 'borough':'Manhattan'})



124

In [34]:
collection.distinct('cuisine')

['Hamburgers',
 'Irish',
 'Jewish/Kosher',
 'American',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Bakery',
 'Chicken',
 'Caribbean',
 'Turkish',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Pizza',
 'Continental',
 'Italian',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'Polish',
 'German',
 'Steak',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'African',
 'Not Listed/Not Applicable',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japanese',
 'Filipino',
 'Juice, Smoothies, Fruit Salads',
 'Brazilian',
 'Afghan'

### Sub-documents

A valid JSON style "document" can have another JSON document inside it.  To access these, we use the "dot" notation to access them.  For example, to get all the restaurants in a certain zipcode, you would run code as follows:

In [38]:
from pprint import pprint
cursor = collection.find({'address.zipcode': '10462'}, {'address.zipcode': 1, '_id':0}).limit(5)
for c in cursor:
    pprint(c)

{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}


### Operators

MongoDB has a series of [operators](https://docs.mongodb.com/manual/reference/operator/query/) which allow us to do more sophisticated filters on our queries.  There are too many to go into individually, but we will look at a few important ones.  The specific syntax varies depending on the operator, so it isn't possible to give a general rule, but we will go over a few examples here.  Make sure you check the [documentation](https://docs.mongodb.com/manual/reference/operator/query/) for use on each one.

#### [\$or](https://docs.mongodb.com/manual/reference/operator/query/or/#op._S_or)

Performs a logical **OR** operation on all the key/value pairs in a list, as in the code below:

In [41]:
filter = {"$or": [{"cuisine": "Polynesian"}, {"cuisine": "Hawaiian"}]}
for f in collection.find(filter, {'name': 1, '_id': 0}).limit(2):
    pprint(f)

{'name': 'Camaradas El Barrio'}
{'name': 'Makana'}


#### [`$regex`](https://docs.mongodb.com/manual/reference/operator/query/regex/#op._S_regex)

The `$regex` operator searches for a regular expression on a particular field.  Within the filter field, the named field (a key) takes a dict as a value.  

For example, to search for all restaurants which start with the word "Pretzel" in the title you can do the following:

In [42]:
filter = {"name": {"$regex": '^Pretzel'}}
collection.count_documents(filter)

4

There are other ways to use regular expressions in PyMongo, you can use the [`re`](https://docs.python.org/3/library/re.html) module in Python.  In Mongo itself, you can use the following syntax: The simplest is to enclose the regular expression inside `/` characters, as in `{"name": /^Pretzel/}`, but that doesn't work properly in PyMongo.

Using the `$regex` operator, find all restaurants which end in "Bar" in the borough of Brooklyn.

HINT: The regex character for the end of a string is `$`

In [44]:
# YOUR CODE HERE
filter_regex = {"name": {"$regex": "Bar$"}}

collection.count_documents(filter_regex)

653

#### [`$gt`](https://docs.mongodb.com/manual/reference/operator/query/gt/#op._S_gt)

The `$gt` operator is a comparison between two values where one is greater than the other.  

For example, consider this code which finds restaurants which have had a score of more than 15:

In [ ]:
filter = {'grades.score': {'$gt': 12}}
collection.count_documents(filter)

Using one of the other comparison operators, find all restaurants which had a grade awarded on the 15 December 2012.  You'll need to create a [`datetime`](https://docs.python.org/2/library/datetime.html#datetime-objects) object in Python.

In [53]:
# YOUR CODE HERE

import datetime

date_to_filter = datetime.datetime(2012, 12, 15)

for doc in collection.find({'grades.date': date_to_filter}, {'_id': 0, 'grades': 1, 'name': 1}):
    pprint (doc)

{'grades': [{'date': datetime.datetime(2014, 8, 5, 0, 0),
             'grade': 'A',
             'score': 10},
            {'date': datetime.datetime(2014, 2, 5, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2012, 12, 15, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2011, 11, 23, 0, 0),
             'grade': 'A',
             'score': 12}],
 'name': 'Ajanta India'}
{'grades': [{'date': datetime.datetime(2014, 12, 16, 0, 0),
             'grade': 'A',
             'score': 4},
            {'date': datetime.datetime(2013, 12, 23, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2012, 12, 15, 0, 0),
             'grade': 'A',
             'score': 10}],
 'name': 'Internationalhouse'}
{'grades': [{'date': datetime.datetime(2015, 1, 14, 0, 0),
             'grade': 'A',
             'score': 6},
            {'date': datetime.datetime

## Organising output

So far, we have seen the two of the arguments in the `find()` and related functions.  The `filter` which allows us to select the criteria for documents in the collection, and the `limit` to limit the amount of results.  You should read the documentation fully about the function in your own time, but for now, we will go over two other arguments which are for organising output: field selection, and sorting.

The field selection or `projection` argument is the argument after the \[optional\] filter, and is either:

* A list of fields to include (plus \_id)
* A dict of fields with True/False to include

For example, to display only the name of the restaurant:

In [54]:
filter = {'cuisine': 'Brazilian'}
fields = {'_id': False, 'name': True}
collection.find_one(filter, fields)

{'name': 'Churrascaria Plataforma'}

The sort argument is a dict object of field names as keys, and directions.  This can be done either as a named parameter when calling `find()`, or as a function in its own right [`sort()`](https://api.mongodb.com/python/current/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort)

For example, to sort in alphabetical order, consider the following code:

In [55]:
import pymongo
# The ASCENDING and DESCENDING constants have values of 1 (ASCENDING) and -1 (DESCENDING)
sort = [('name', pymongo.ASCENDING)]
for d in collection.find(filter, projection=fields, sort=sort):
    pprint(d)

{'name': 'Barzinho'}
{'name': 'Beco'}
{'name': 'Beija-Flor'}
{'name': 'Berimbau'}
{'name': 'Brazil Brazil Restuarant'}
{'name': 'Brazil Grill'}
{'name': 'Buffet 58'}
{'name': 'Carioca Grill'}
{'name': 'Casa'}
{'name': 'Churrascaria Plataforma'}
{'name': 'Circus Restaurant'}
{'name': 'Copacabana Pizza & Grill'}
{'name': 'Emporium Brasil Restaurant'}
{'name': 'Esperanto'}
{'name': 'Fogo De Chao'}
{'name': 'Malagueta  Restaurant'}
{'name': 'Miss Favela'}
{'name': 'Pacificos Fine Foods'}
{'name': 'Point Brazil'}
{'name': 'Rainhas'}
{'name': 'Rice & Beans Restaurant'}
{'name': 'Sao Restaurant'}
{'name': 'Texas De Brazil Churrascaria'}
{'name': 'Via Brazil'}
{'name': 'Villa Brazil Cafe Grill'}
{'name': 'Zebu Grill'}


# MongoDB Aggregation Framework

The most common usage for the aggregation framework is to perform group operations such as sum, count or average.  The framework works as a pipeline, with a series of different stages where the data are transformed in each one.

At its simplest, this can be used to obtain output like min, max, count, avg on a collection as follows:

[link](https://docs.mongodb.com/manual/reference/operator/aggregation/group/)

In [56]:
group = {
    '$group': {
        '_id': None, 
        'size': {'$sum': 1},
        'min': {'$min': '$restaurant_id'},
        'max': {'$max': '$restaurant_id'}
    }
}

cursor = collection.aggregate([group])
for c in cursor:
    print(c)

{'_id': None, 'size': 25359, 'min': '30075445', 'max': '50018995'}


Note that it has an `$_id: None` key/value pair in it.  It is compulsory for a `$group` pipeline to have one, and it indicates what it is grouping by.  In this case, we haven't grouped it at all, however it can also be used for more complex output where documents are grouped according to a field.

### Aggregation example

Consider this example, of finding the breakdown of how many of each type of restaurant there is in the Bronx.  We would need to go through the following stages:

- Identify restaurants which are in the Bronx
- Group the restaurants by type to get the count
- Sort the results in a sensible way

The code to perform this query is below:

In [60]:
# Restrict the results to only establishments in the Bronx.  
# '$match' indicates the stage in the pipeline, and the dictionary is the same as using with find()
match = {
    "$match": {"borough": "Bronx"}
}

# $group indicates the stage in the pipeline
# _id is the field to perform the operation on (like SQL GROUP BY)
# count is the name of the field that the result will be in
# $sum is the counting operation, and the value 1 is how many to count each time
group = {
    '$group': {'_id': '$cuisine', 'count': {'$sum': 1}}
    
}
# $sort indicates the position in the pipeline
# count is the field to sort by, and -1 means to sort in descending order
sort = {
    '$sort': {'count': pymongo.DESCENDING}
}

match_final = {
    "$match": {"count": {"$gt": 100}}
}

cursor = collection.aggregate([match, group, sort, match_final])
for c in cursor:
    print(c)



{'_id': 'American', 'count': 411}
{'_id': 'Chinese', 'count': 323}
{'_id': 'Pizza', 'count': 197}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 187}
{'_id': 'Spanish', 'count': 127}
{'_id': 'Caribbean', 'count': 110}
{'_id': 'Chicken', 'count': 108}


This is a simple query, which shows some of the basic stages of the aggregation pipeline.  It can be improved as follows:

* We can change the name of the `_id` in the output back to `cuisine` using the `$project` stage
* We can change the order of the output to be sorted in alphabetical order as well
* We can limit the results to include results only with a count of 20 or more

Implement those stages in the cell below

In [68]:
# YOUR CODE HERE

project_stage = {
    "$project": {"cuisine": "$_id", "count": 1}
}

sort_stage = {
    "$sort": {"cuisine": pymongo.ASCENDING}
}

match_stage = {
    "$match": {"count": {"$gt":20} }
}

pipeline = [match, group, project_stage, match_stage, sort_stage] #More code here...
cursor = collection.aggregate(pipeline)
for c in cursor:
    print(c)


{'_id': 'African', 'count': 31, 'cuisine': 'African'}
{'_id': 'American', 'count': 411, 'cuisine': 'American'}
{'_id': 'Bakery', 'count': 71, 'cuisine': 'Bakery'}
{'_id': 'Café/Coffee/Tea', 'count': 45, 'cuisine': 'Café/Coffee/Tea'}
{'_id': 'Caribbean', 'count': 110, 'cuisine': 'Caribbean'}
{'_id': 'Chicken', 'count': 108, 'cuisine': 'Chicken'}
{'_id': 'Chinese', 'count': 323, 'cuisine': 'Chinese'}
{'_id': 'Delicatessen', 'count': 26, 'cuisine': 'Delicatessen'}
{'_id': 'Donuts', 'count': 68, 'cuisine': 'Donuts'}
{'_id': 'Hamburgers', 'count': 78, 'cuisine': 'Hamburgers'}
{'_id': 'Ice Cream, Gelato, Yogurt, Ices', 'count': 27, 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices'}
{'_id': 'Italian', 'count': 52, 'cuisine': 'Italian'}
{'_id': 'Juice, Smoothies, Fruit Salads', 'count': 35, 'cuisine': 'Juice, Smoothies, Fruit Salads'}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 187, 'cuisine': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)

### Detour: 

In [ ]:
# YOUR CODE HERE FOR sort

pipeline.append(sort)
cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


In [ ]:
# YOUR CODE HERE FOR `count` > 20

cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


## Challenge

How would you work out the percentage of each type of cuisine out of all selected restaurants?

In [ ]:
# YOUR CODE HERE...

# Where next?

* Dealing with array data
* Update, delete, and drop
* Setting up authentication
* Sharding databases